In [1]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.naive_bayes
import sklearn.ensemble as ske
import sklearn.neighbors
import sklearn.pipeline
import sklearn.linear_model
import joblib
from sklearn import preprocessing
from sklearn import tree, linear_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectFromModel
import sys
import sqlite3
import importlib

In [2]:
df_wild = pd.read_pickle('/kaggle/input/wild-pickle/wild.pickle')

In [3]:
MAX_LENGTH = int(1e6)
prev_size = len(df_wild)
df_wild = df_wild[df_wild['generic_fileSize'] <= MAX_LENGTH]
print("considering samples with size less than {} bytes, {} ----> {}".format(MAX_LENGTH, prev_size, len(df_wild)))

considering samples with size less than 1000000 bytes, 50724 ----> 37354


In [4]:
print(df_wild.shape)

(37354, 56555)


In [5]:
df_wild.head()

,sample_sha1,benign,malicious,packed,unpacked_sample_sha1,unpacked_sample_id,packer_name,imp_setupremovefilelogentrya,imp_vardeccmp,imp_setdlgitemint,...,opcode_xor sub mov,opcode_xor sub mov add,opcode_xor test,opcode_xor test je,opcode_xor xor,opcode_xor xor mov,opcode_xor xor pop,opcode_xor xor pop mov,opcode_xor xor xor,opcode_xor xor xor pop
1,a2aac23e17be570c647fd22080e05b0e58449565,True,False,True,-1,-1,none,False,False,False,...,0.000022,0.0,0.000088,0.000084,0.000070,0.000066,0.000028,0.0,0.000053,0.0
2,c3aa534bbd20f3a3dd8e7a457f625291af9d0fe4,False,True,True,-1,-1,none,False,False,False,...,0.000000,0.0,0.000000,0.000000,0.000309,0.000000,0.000000,0.0,0.000331,0.0
10,15cb9eab6b0f0ccd718a9235f07046dff100789f,False,True,True,-1,-1,none,False,False,False,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
12,8792cf08950abada1572b47948e86c9fce26c01a,True,False,True,-1,-1,none,False,False,False,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
13,353a955ae9919f7b639d74394e3c1149f0189362,True,False,True,-1,-1,none,False,False,False,...,0.000000,0.0,0.000204,0.000190,0.000066,0.000091,0.000000,0.0,0.000000,0.0


Chia nhỏ dataset

In [6]:
#Chọn ngẫu nhiên 12,000 mẫu
df = df_wild.sample(n=12000, random_state=102)

Xử lý dataset

In [7]:
# Loại bỏ các lables không cần thiết
df = df.drop(['sample_sha1','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
df = df.drop(['benign_vt','malicious_vt'], axis=1)
df = df.drop(['benign','packer_name','source','similarity','most_similar_sha1','unpacked_similarity'], axis=1)

In [8]:
# Xử lý các cột có dạng object
for col in df.columns:
    dtype = df[col].dtype
    if dtype == object:
        le = preprocessing.LabelEncoder()
        df[col] = le.fit_transform(df[col])

In [9]:
df.head()

,malicious,packed,imp_setupremovefilelogentrya,imp_vardeccmp,imp_setdlgitemint,imp_?append@?$basic_string@du?$char_traits@d@std@@v?$allocator@d@2@@std@@qaeaav12@abv12@ii@z,imp_zwwritefile,imp_glstencilop,imp_comparelinkinforeferents,imp__setmbcp,...,opcode_xor sub mov,opcode_xor sub mov add,opcode_xor test,opcode_xor test je,opcode_xor xor,opcode_xor xor mov,opcode_xor xor pop,opcode_xor xor pop mov,opcode_xor xor xor,opcode_xor xor xor pop
32148,False,True,False,False,False,False,False,False,False,False,...,0.0,0.0,0.000187,0.000198,0.0,0.0,0.0,0.0,0.0,0.0
210953,False,False,False,False,False,False,False,False,False,False,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
50313,False,True,False,False,False,False,False,False,False,False,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
200288,True,True,False,False,False,False,False,False,False,False,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
49579,True,True,False,False,False,False,False,False,False,False,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
print(df.shape)

(12000, 56544)


Chia dataset thành 10000 tập để train và 2000 mẫu để test

In [11]:
df_train = df.sample(n=10000, random_state=102)
df_test = df.drop(df_train.index)

In [12]:
#Vì tập dữ liệu không có Unpacked Malware, chỉ cần loại bỏ Packed Benign
data = df_train.drop(df_train[(df_train['malicious'] == False) & (df_train['packed'] == True)].index)

In [13]:
#Kiểm tra số lượng mẫu
Unpacked_Benign = data[(data['malicious'] == False) & (data['packed'] == False)].shape[0]
Packed_Malware = data[(data['malicious'] == True) & (data['packed'] == True)].shape[0]
print('Unpacked Benign: %i samples' % Unpacked_Benign)
print('Packed Malware: %i samples' % Packed_Malware)
print('Total TRAIN set: %i samples'% (Unpacked_Benign+Packed_Malware))


Unpacked Benign: 662 samples
Packed Malware: 7535 samples
Total TRAIN set: 8197 samples


In [14]:
#Chia tỉ lệ 50 - 50
data_PM = data[(data['malicious'] == True) & (data['packed'] == True)].sample(n=Packed_Malware-Unpacked_Benign, random_state=102)
data = data.drop(data_PM.index)

In [15]:
#Số lượng tập train
Unpacked_Benign = data[(data['malicious'] == False) & (data['packed'] == False)].shape[0]
Packed_Malware = data[(data['malicious'] == True) & (data['packed'] == True)].shape[0]
print('Unpacked Benign: %i samples' % Unpacked_Benign)
print('Packed Malware: %i samples' % Packed_Malware)
print('Total TRAIN set: %i samples'% (Unpacked_Benign+Packed_Malware))

Unpacked Benign: 662 samples
Packed Malware: 662 samples
Total TRAIN set: 1324 samples


In [16]:
#Xử lý tập train
y = data['malicious'].values
data = data.drop(['malicious','packed'], axis=1)

Xử lý tập test

In [17]:
Unpacked_Benign = df_test[(df_test['malicious'] == False) & (df_test['packed'] == False)].shape[0]
Packed_Malware = df_test[(df_test['malicious'] == True) & (df_test['packed'] == True)].shape[0]
Packed_Benign = df_test[(df_test['malicious'] == False) & (df_test['packed'] == True)].shape[0]

df_test_PM = df_test[(df_test['malicious'] == True) & (df_test['packed'] == True)].sample(n=(Packed_Malware-(Unpacked_Benign+Packed_Benign)), random_state=102)
df_test = df_test.drop(df_test_PM.index)

Unpacked_Benign = df_test[(df_test['malicious'] == False) & (df_test['packed'] == False)].shape[0]
Packed_Malware = df_test[(df_test['malicious'] == True) & (df_test['packed'] == True)].shape[0]
Packed_Benign = df_test[(df_test['malicious'] == False) & (df_test['packed'] == True)].shape[0]

print('Unpacked Benign: %i samples' % Unpacked_Benign)
print('Packed Malware: %i samples' % Packed_Malware)
print('Packed Benign: %i samples' % Packed_Benign)
print('Total TEST set: %i samples'% (Unpacked_Benign+Packed_Malware+Packed_Benign))

Unpacked Benign: 142 samples
Packed Malware: 501 samples
Packed Benign: 359 samples
Total TEST set: 1002 samples


In [18]:
#Chọn lọc features
print('Researching important feature based on %i total features' % data.shape[1])
fsel = ske.ExtraTreesClassifier().fit(data, y)

Researching important feature based on 56542 total features


In [19]:
#Tạo mô hình và chia tập train - test
model = SelectFromModel(fsel, prefit=True)

X_train = model.transform(data)
X_test = model.transform(df_test.drop(['malicious','packed'], axis=1))
y_train = y
y_test = df_test['malicious'].values

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [20]:
#Tạo mô hình và chia tập train - test
model = SelectFromModel(fsel, prefit=True)

X_train = data
X_test = df_test.drop(['malicious','packed'], axis=1)
y_train = y
y_test = df_test['malicious'].values

In [21]:
nb_features = X_train.shape[1]
features = []

indices = np.argsort(fsel.feature_importances_)[::-1][:nb_features]

In [22]:
#Gán vào tập features để xử lý mô hình
for f in sorted(np.argsort(fsel.feature_importances_)[::-1][:nb_features]):
    features.append(data.columns[f])

In [23]:
algorithms = {
        "DecisionTree": tree.DecisionTreeClassifier(max_depth=20),
        "RandomForest": ske.RandomForestClassifier(n_estimators=100),
        "GradientBoosting": ske.GradientBoostingClassifier(n_estimators=100),
        "AdaBoost": ske.AdaBoostClassifier(n_estimators=200)
    }

results = {}
print("Now testing algorithms")
for algo in algorithms:
    clf = algorithms[algo]
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print("%s : %f %%" % (algo, score*100))
    results[algo] = score
    res = clf.predict(X_test)
    mt = confusion_matrix(y_test, res)
    report = classification_report(y_test, res)
    print("True positive : %i " % (mt[1][1]))
    print("True negative : %i " % (mt[0][0]))
    print("False positive : %i " % (mt[0][1]))
    print("False negative : %i " % (mt[1][0]))
    print("Classification Report:\n", report)
    print("\n")

Now testing algorithms
DecisionTree : 82.335329 %
True positive : 457 
True negative : 368 
False positive : 133 
False negative : 44 
Classification Report:
               precision    recall  f1-score   support

       False       0.89      0.73      0.81       501
        True       0.77      0.91      0.84       501

    accuracy                           0.82      1002
   macro avg       0.83      0.82      0.82      1002
weighted avg       0.83      0.82      0.82      1002



RandomForest : 82.834331 %
True positive : 464 
True negative : 366 
False positive : 135 
False negative : 37 
Classification Report:
               precision    recall  f1-score   support

       False       0.91      0.73      0.81       501
        True       0.77      0.93      0.84       501

    accuracy                           0.83      1002
   macro avg       0.84      0.83      0.83      1002
weighted avg       0.84      0.83      0.83      1002



GradientBoosting : 85.329341 %
True positive : 

In [24]:
#Chọn mô hình có Acc lớn nhất
winner = max(results, key=results.get)
print('Algorithm with highest accuracy on train/test is %s with a %f %% success' % (winner, results[winner]*100))

Algorithm with highest accuracy on train/test is GradientBoosting with a 85.329341 % success
